# Python Pandas 4 -- Working w/ Multiple Data Frames

We looked at merging briefly in the previous examples (mostly because the data I selected called for it!). Here we're going to look at something a little bit more focused. 

### Setup
Let's start by setting up our environment

In [2]:
# pandas 
# in many examples and codebases you'll find statements that alias the import (i.e import pandas as pd). 
# I don't like doing this due to the ambiguity. Autocomplete protects our fingers. 
import pandas

In [3]:
# orders
orders = pandas.read_csv('orders.csv')
orders

,order_id,customer_id,product_id,quantity,timestamp
0,1,2,3,1,2024-04-01
1,2,2,2,3,2024-04-01
2,3,3,1,1,2024-04-01
3,4,3,2,2,2024-05-01
4,5,3,3,3,2024-05-01
5,6,1,4,2,2024-06-01
6,7,1,1,1,2024-05-02
7,8,1,4,1,2024-05-02


In [4]:
# customers
customers = pandas.read_csv('customers.csv')
customers

,customer_id,customer_name,address,phone_number
0,1,Sherlock Holmes,221B Baker St.,020 7224 3688
1,2,Sam Malone,112 1/2 Beacon St.,617-227-9605
2,3,Homer Simpson,742 Evergreen Terrace,123-555-0113


In [5]:
# products
products = pandas.read_csv('products.csv')
products

,id,description,price
0,1,Apple iPhone,1099
1,2,Nintendo Switch,299
2,3,Amazon Echo Show 10,249
3,4,Google Home Max,399


In [6]:
# monthly_sales
sales = pandas.read_csv('monthly_sales.csv')
sales

,month,revenue
0,January,300
1,February,290
2,March,310
3,April,325
4,May,475
5,June,495


In [7]:
# monthly sales targets
targets = pandas.read_csv('monthly_targets.csv')
targets

,month,target
0,January,310
1,February,270
2,March,300
3,April,350
4,May,475
5,June,500


In [8]:
# This is a comparison of east vs. west regions 
east_vs_west = pandas.read_csv('east_vs_west.csv')
east_vs_west

,month,west,east
0,January,30,35
1,February,29,35
2,March,31,29
3,April,32,28
4,May,47,50
5,June,49,45


### Inner Merge

Let's just create a basic inner merge. (This is the default merge, so we don't have to specify it, but if we decide to do it anyway it would be **how='inner'**

In [10]:
# comparing sales and targets
sales_and_targets = pandas.merge(sales, targets)
sales_and_targets

,month,revenue,target
0,January,300,310
1,February,290,270
2,March,310,300
3,April,325,350
4,May,475,475
5,June,495,500


In [11]:
# We can do different things like calculate the difference
sales_and_targets['difference'] = sales_and_targets['revenue'] - sales_and_targets['target']
sales_and_targets

,month,revenue,target,difference
0,January,300,310,-10
1,February,290,270,20
2,March,310,300,10
3,April,325,350,-25
4,May,475,475,0
5,June,495,500,-5


In [12]:
# ... or separate out the months we're over 
sales_and_targets[sales_and_targets.difference > 0]

,month,revenue,target,difference
1,February,290,270,20
2,March,310,300,10


In [13]:
# (We didn't need to create a new column to do this...) 
sales_and_targets[sales_and_targets.revenue > sales_and_targets.target]

,month,revenue,target,difference
1,February,290,270,20
2,March,310,300,10


Our previous merges have been performed on DataFrame's merge() method, however each instance of the DataFrame has it's own merge() method. This is the preferential method for merging dataframes when you are going to chain more than 2 dataframes together. (Basic Chain of Responsibility). 

In [15]:
# let's chain our sales data
chained_sales = sales.merge(targets).merge(east_vs_west)
chained_sales

,month,revenue,target,west,east
0,January,300,310,30,35
1,February,290,270,29,35
2,March,310,300,31,29
3,April,325,350,32,28
4,May,475,475,47,50
5,June,495,500,49,45


In [16]:
# we can now perform complex filters on multiple data frames
chained_sales[(chained_sales.revenue > chained_sales.target) & (chained_sales.east > chained_sales.west)]

,month,revenue,target,west,east
1,February,290,270,29,35


### Merging on specific columns

Prior to this, the examples automatically merged on the columns that made sense, because there was only a single shared column name. This is a neat feature of pandas. (That can be painful when working w/ many datasets that you aren't familiar with. **ALWAYS INSPECT YOUR DATA**

Its far more common that you'll find data frames that don't have shared column names (or if they do, the column names don't have the same semantics)

In [18]:
# Lets take a look at our orders and product data frames. I know, we looked at them above.. 
print(orders)
print(products)

   order_id  customer_id  product_id  quantity   timestamp
0         1            2           3         1  2024-04-01
1         2            2           2         3  2024-04-01
2         3            3           1         1  2024-04-01
3         4            3           2         2  2024-05-01
4         5            3           3         3  2024-05-01
5         6            1           4         2  2024-06-01
6         7            1           1         1  2024-05-02
7         8            1           4         1  2024-05-02
   id          description  price
0   1         Apple iPhone   1099
1   2      Nintendo Switch    299
2   3  Amazon Echo Show 10    249
3   4      Google Home Max    399



We have no common column names in this case, but we have a foreign key relationship between **orders.product_id** and **products.id**. This means we have a way to merge the two frames... but how?  

The simplest solution would be to make the names match so that pandas can work its black magic!

In [20]:
# Solution 1: Trust the Black Magic -- rename your columns. 
orders_and_products = pandas.merge(
    orders,
    products.rename(columns={'id':'product_id'})
)
orders_and_products

,order_id,customer_id,product_id,quantity,timestamp,description,price
0,1,2,3,1,2024-04-01,Amazon Echo Show 10,249
1,2,2,2,3,2024-04-01,Nintendo Switch,299
2,3,3,1,1,2024-04-01,Apple iPhone,1099
3,4,3,2,2,2024-05-01,Nintendo Switch,299
4,5,3,3,3,2024-05-01,Amazon Echo Show 10,249
5,6,1,4,2,2024-06-01,Google Home Max,399
6,7,1,1,1,2024-05-02,Apple iPhone,1099
7,8,1,4,1,2024-05-02,Google Home Max,399


In [21]:
# Solution 2: A more SQL-like solution -- use merge, but specify the column relationship
## Suffixes are ways for pandas to resolve columns w/ the same name. It won't allow this. 
## The suffixes tell you where they've originated from (the defaults are _x and _y)
orders_and_products = pandas.merge(
    orders,
    products,
    left_on='product_id',
    right_on='id',
    suffixes=['_orders','_products'])
orders_and_products

,order_id,customer_id,product_id,quantity,timestamp,id,description,price
0,1,2,3,1,2024-04-01,3,Amazon Echo Show 10,249
1,2,2,2,3,2024-04-01,2,Nintendo Switch,299
2,3,3,1,1,2024-04-01,1,Apple iPhone,1099
3,4,3,2,2,2024-05-01,2,Nintendo Switch,299
4,5,3,3,3,2024-05-01,3,Amazon Echo Show 10,249
5,6,1,4,2,2024-06-01,4,Google Home Max,399
6,7,1,1,1,2024-05-02,1,Apple iPhone,1099
7,8,1,4,1,2024-05-02,4,Google Home Max,399


Let's show how merges go wrong... (let's look at products and orders again)

In [23]:
# I lied, let's look at a different example of orders
orders2 = pandas.read_csv('orders2.csv')
orders2

,order_id,customer_id,product_id,quantity,timestamp
0,1,2,3,1,2024-04-01
1,2,2,2,3,2024-04-01
2,3,3,5,1,2024-04-01
3,4,3,2,2,2024-05-01
4,5,3,3,3,2024-05-01
5,6,1,4,2,2024-06-01
6,7,1,1,1,2024-05-02
7,8,1,4,1,2024-05-02


In [24]:
products

,id,description,price
0,1,Apple iPhone,1099
1,2,Nintendo Switch,299
2,3,Amazon Echo Show 10,249
3,4,Google Home Max,399


Did you notice that there is a **product_id=5** that corresponds to **order_id=3**?? 
However, there that **product_id** doesn't exist in the product dataframe.

Let's do a default inner merge to see waht happens

In [26]:
busted_merge = pandas.merge(
    orders2,
    products,
    left_on="product_id",
    right_on="id"
)
busted_merge

,order_id,customer_id,product_id,quantity,timestamp,id,description,price
0,1,2,3,1,2024-04-01,3,Amazon Echo Show 10,249
1,2,2,2,3,2024-04-01,2,Nintendo Switch,299
2,4,3,2,2,2024-05-01,2,Nintendo Switch,299
3,5,3,3,3,2024-05-01,3,Amazon Echo Show 10,249
4,6,1,4,2,2024-06-01,4,Google Home Max,399
5,7,1,1,1,2024-05-02,1,Apple iPhone,1099
6,8,1,4,1,2024-05-02,4,Google Home Max,399


????
Where did order 3 go??

Ok. Ok. You got me. Inner merge is the same concept as an inner join, so the "query" (or merge in this case) is only going to include rows that have complete/perfect matches. 

Let's look at two different locations of "Sully's Hahdware" stores in the Greater Boston area. (Yes, I made it up).

In [28]:
# Billerica store
billerica = pandas.read_csv('billerica.csv')
billerica

,item,billerica_store_inventory
0,hammah,12
1,screwdrivah,15
2,nails,200
3,screws,350
4,cah chahgah,6
5,duct tape,150
6,wrench,12
7,pvc pipe,54


In [29]:
# Methuen store
methuen = pandas.read_csv('methuen.csv')
methuen

,item,methuen_store_inventory
0,hammah,6
1,nails,250
2,cah chahgah,6
3,duct tape,150
4,pvc pipe,54
5,rake,10
6,shovel,15
7,wooden dowels,192


As you can see there are a lot of columns without perfect matches, so an inner merge probably isn't going to work. Introducing

### Outer Merges!

An outer merge includes ALL rows from both tables even if they don't match. (remember?? We used this strategy w/ the Celtics!) 

In [31]:
pandas.merge(billerica, methuen, how='outer')

,item,billerica_store_inventory,methuen_store_inventory
0,cah chahgah,6.0,6.0
1,duct tape,150.0,150.0
2,hammah,12.0,6.0
3,nails,200.0,250.0
4,pvc pipe,54.0,54.0
5,rake,NaN,10.0
6,screwdrivah,15.0,NaN
7,screws,350.0,NaN
8,shovel,NaN,15.0
9,wooden dowels,NaN,192.0


### Left and Right Merge

As we saw above, imperfect matched rows result in NaNs or Nones. In the case above, it is a valid value, because we're measuring inventory across multiple stores. 

Let's look at left and right merge. 

In [33]:
# left merge for Billerica
## This is going to show the products that are in Billerica but not Methuen. 
## In other words we'll only include non-matching values from the "right".
pandas.merge(billerica, methuen, how='left')

,item,billerica_store_inventory,methuen_store_inventory
0,hammah,12,6.0
1,screwdrivah,15,NaN
2,nails,200,250.0
3,screws,350,NaN
4,cah chahgah,6,6.0
5,duct tape,150,150.0
6,wrench,12,NaN
7,pvc pipe,54,54.0


In [34]:
# Lets swap this as a left merge for methuen  (Same concept, just switching which column is on the left)
pandas.merge(methuen, billerica, how='left') 

,item,methuen_store_inventory,billerica_store_inventory
0,hammah,6,12.0
1,nails,250,200.0
2,cah chahgah,6,6.0
3,duct tape,150,150.0
4,pvc pipe,54,54.0
5,rake,10,NaN
6,shovel,15,NaN
7,wooden dowels,192,NaN


In [35]:
# What about a right merge??
## This will show the same result we had when performing a left merge on billerica, but we've reordered the columns. 
pandas.merge(methuen, billerica, how='right')

,item,methuen_store_inventory,billerica_store_inventory
0,hammah,6.0,12
1,screwdrivah,NaN,15
2,nails,250.0,200
3,screws,NaN,350
4,cah chahgah,6.0,6
5,duct tape,150.0,150
6,wrench,NaN,12
7,pvc pipe,54.0,54


In [36]:
# right merge on methuen to go full circle...
pandas.merge(billerica, methuen, how='right')

,item,billerica_store_inventory,methuen_store_inventory
0,hammah,12.0,6
1,nails,200.0,250
2,cah chahgah,6.0,6
3,duct tape,150.0,150
4,pvc pipe,54.0,54
5,rake,NaN,10
6,shovel,NaN,15
7,wooden dowels,NaN,192


### Concatenation

This is a wonderful tool. Concatenate allows you to add multiple dataframes together. This is very useful when you want to store large (**LARGE**) datasets in the cloud, but want to avoid massive download times. It's also useful in breaking up spreadsheets or csv files to avoid LFS limitations in git. 

In [72]:
# go get our CSVs.
black_lion = pandas.read_csv('black_lion.csv')
red_lion = pandas.read_csv('red_lion.csv')
blue_lion = pandas.read_csv('blue_lion.csv')
green_lion = pandas.read_csv('green_lion.csv')
yellow_lion = pandas.read_csv('yellow_lion.csv')

voltron = pandas.concat([black_lion, red_lion,blue_lion,green_lion,yellow_lion]).reset_index()
voltron

,index,pilot,og_voice_actor,recent_voice_actor
0,0,"Takashi ""Shiro"" Shirogane",NaN,Josh Keaton
1,1,Sven Holgersson,Michael Bell,NaN
2,2,Keith,Neil Ross,Steven Yeun
3,3,Zarkon,Jack Angel,Neil Kaplan
4,4,Shiro's Clone,NaN,Josh Keaton
5,5,"Akira ""Chief"" Kogane",Kazuhiko Inoue,NaN
6,0,Keith,Neil Ross,Steven Yeun
7,1,Lance,Michael Bell,Jeremy Shada
8,2,Alfor,Peter Cullen,Keith Ferguson Sean Teale
9,3,"Isamu ""Moody"" Kurogane",Yu Mizushima,NaN
